# SMU-MSDS-7337-Natural Language Processing HomeWork 5

## By-Rashmi Patel

In [1]:
# Environment Prep
import bs4; print( 'bs4 ' + bs4.__version__)
from bs4 import BeautifulSoup, SoupStrainer

import nltk; print( 'nltk ' + nltk.__version__)
from nltk import word_tokenize, pos_tag, RegexpParser;
from nltk.tokenize import sent_tokenize

import re; print('re ' + re.__version__)
import requests; print('requests ' + requests.__version__)



bs4 4.9.3
nltk 3.6.1
re 2.2.1
requests 2.25.1


In [2]:
import requests
import re
from bs4 import BeautifulSoup
from urllib import request
# !pip install IMDbPy
import imdb
import pandas as pd
import numpy as np

## HW 5: Question 1
Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.

* It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.

* Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.

* Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.

* Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.

In [3]:
rotw_url='https://www.imdb.com/title/tt8179388/reviews/?ref_=tt_ql_urv'
aladin_url='https://www.imdb.com/title/tt6139732/reviews/?ref_=tt_ql_urv'
mm_url='https://www.imdb.com/title/tt1618434/reviews/?ref_=tt_ql_urv'
hustle_url='https://www.imdb.com/title/tt1298644/reviews/?ref_=tt_ql_urv'
tsop_url='https://www.imdb.com/title/tt5113040/reviews/?ref_=tt_ql_urv'
tfj='https://www.imdb.com/title/tt5235880/reviews/?ref_=tt_ql_urv'

In [4]:
# Function to extract static links
def find_permalinks(url):
    page = request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    review_containers = soup.find_all('div', class_ = 'review-container')
    for review in review_containers:
        permalinks.append("https://www.imdb.com" + review.find('a', attrs={'href': re.compile("/review")}).get('href'))
        


In [5]:
# Storing all static links obtained in an array
permalinks = []

find_permalinks(rotw_url)
find_permalinks(aladin_url)
find_permalinks(mm_url)
find_permalinks(hustle_url)
find_permalinks(tsop_url)
find_permalinks(tfj)

print(len(permalinks))

150


### Storing the reviews in an array


In [9]:
%%time
reviews = []


for link in permalinks:
    page = request.urlopen(link)
    soup = BeautifulSoup(page, 'html.parser')
    review_containers = soup.find_all('div', class_ = 'review-container')
    for review in review_containers:
        reviews.append(review.find(class_ = 'text show-more__control').text)
print(len(reviews))

150
CPU times: user 9.45 s, sys: 503 ms, total: 9.95 s
Wall time: 1min 11s


In [10]:
# test code to extract class names

# for link in permalinks:
#     page = request.urlopen(link)
#     soup = BeautifulSoup(page, 'html.parser')
#     review_containers = soup.find_all('div', class_ = 'review-container')
#     print(review_containers)
#     print('**********************+++++++++++++++++++++++++')

### Storing the movie names in an array 


In [11]:
%%time
movie_names = []


for link in permalinks:
    page = request.urlopen(link)
    soup = BeautifulSoup(page, 'html.parser')
    title_containers = soup.find_all('div', class_ = 'review-container')
    for review in title_containers:
        movie_names.append(review.find(class_ = 'lister-item-header').text)
print(len(movie_names))

150
CPU times: user 8.25 s, sys: 398 ms, total: 8.64 s
Wall time: 1min 11s


### Removing the newline \n from movie names


In [12]:
movie=[]
for names in movie_names:
    names=names.strip()
    names=re.sub('\n'," ",names)
    movie.append(names)

print(len(movie))  


150


In [13]:
# Creating s dataframe with movie name, li nk and review
data={'Movie_Name':movie,'Review_URL':permalinks,'Review_Description':reviews}
df=pd.DataFrame(data)
df.head()

,Movie_Name,Review_URL,Review_Description
0,Rim of the World (2019),https://www.imdb.com/review/rw4887915/,Four kids from various background go to summer...
1,Rim of the World (2019),https://www.imdb.com/review/rw5001446/,This is a fun teens save the world film. Three...
2,Rim of the World (2019),https://www.imdb.com/review/rw4886724/,The Babysitter restored my faith in McG after ...
3,Rim of the World (2019),https://www.imdb.com/review/rw5863348/,Four misfit children in summer camp are tasked...
4,Rim of the World (2019),https://www.imdb.com/review/rw5671255/,It is a pre-teen oriented movie about an alien...


## HW5: Question 2 
Extract noun phrase (NP) chunks from your reviews using the following procedure:

* In Python, use BeautifulSoup to grab the main review text from each link.  
* Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 
* You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.



In [14]:
NP_grammar = r"""
  NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
      {<NNP>+}                # chunk sequences of proper nouns
      {<DT>?<JJ.*>*<NN.*>+}
      {<PRP$><NNP>+<NN>}
      {<DT><NN><VBG>}
      {<CD><NN><TO><CD><NN>}
      {<JJR><IN><CD><NN>}
      {<DT><JJS><VBG><NNS>}
"""
tokens = nltk.word_tokenize(reviews[1])
tagged = nltk.pos_tag(tokens)    
entities = nltk.chunk.ne_chunk(tagged)
print(entities)

(S
  This/DT
  is/VBZ
  a/DT
  fun/NN
  teens/NNS
  save/VBP
  the/DT
  world/NN
  film/NN
  ./.
  Three/CD
  diverse/JJ
  kids/NNS
  go/VBP
  to/TO
  camp/VB
  ./.
  They/PRP
  get/VBP
  separated/VBN
  from/IN
  the/DT
  group/NN
  and/CC
  meet/VB
  a/DT
  fourth/JJ
  kid/NN
  hanging/VBG
  out/RP
  in/IN
  the/DT
  woods/NNS
  with/IN
  a/DT
  bizarre/JJ
  life/NN
  story/NN
  ./.
  Then/RB
  the/DT
  alien/NN
  invasion/NN
  happens/VBZ
  ./.
  It/PRP
  also/RB
  happens/VBZ
  that/IN
  an/DT
  astronaut/JJ
  crashes/NNS
  by/IN
  them/PRP
  and/CC
  hands/VBZ
  them/PRP
  a/DT
  key/NN
  that/WDT
  must/MD
  bee/VB
  taken/VBN
  to/TO
  (ORGANIZATION NASA/NNP)
  ,/,
  70/CD
  miles/NNS
  away/RB
  ./.
  They/PRP
  start/VBP
  the/DT
  trek/NN
  with/IN
  aliens/NNS
  on/IN
  their/PRP$
  tail.The/NN
  film/NN
  plot/NN
  lacked/VBD
  ,/,
  but/CC
  the/DT
  personalities/NNS
  of/IN
  the/DT
  kids/NNS
  and/CC
  frequent/JJ
  movie/NN
  references/NNS
  made/VBD
  it/PRP
  great

In [15]:
# Function for chunking the reviews
def review_chunker (review_text) :

    tokens = nltk.word_tokenize(review_text)
    tagged = nltk.pos_tag(tokens)    
    entities = nltk.chunk.ne_chunk(tagged)
    return (entities)

## HW5: Question 3
Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).

In [16]:
# Printing all the chnuked reviews 
for name, review in zip(df['Movie_Name'], df['Review_Description']) :
    
    print ("\n-----------------------********************************------------------")
    print ("\t\t", name)
    print ("-----------------------********************************------------------")
    
    entities = review_chunker(review)

    cp = nltk.RegexpParser(NP_grammar)
    result = cp.parse(entities)

    for subtree in result.subtrees():
        if subtree.label() == 'NP': print(subtree)


-----------------------********************************------------------
		 Rim of the World (2019)
-----------------------********************************------------------
(NP kids/NNS)
(NP various/JJ background/NN)
(NP summer/NN)
(NP camp/NN)
(NP World/NNP)
(NP Aliens/NNS)
(NP the/DT only/JJ hope/NN)
(NP the/DT kids/NNS)
(NP the/DT mystery/NN)
(NP girl/NN)
(NP brash/NN)
(NP the/DT rescue/NN)
(NP the/DT sole/JJ survivor/NN)
(NP Station/NNP)
(NP the/DT secret/JJ code/NN)
(NP the/DT aliens.McG/NN)
(NP cartoon/NN)
(NP character/NN)
(NP broad/JJ characters/NNS)
(NP the/DT rest/NN)
(NP the/DT kids/NNS)
(NP The/DT alien/NN)
(NP an/DT alien/NN)
(NP dog/NN)
(NP something/NN)
(NP sense/NN)
(NP minute/NN)
(NP thing/NN)
(NP any/DT other/JJ forms/NNS)
(NP a/DT blob/NN)
(NP A/DT blob/NN)
(NP a/DT great/JJ traditional/JJ horror/NN)
(NP creature/NN)
(NP Another/DT question/NN)
(NP people/NNS)
(NP guns/NNS)
(NP a/DT bunch/NN)
(NP kids/NNS)
(NP scary/JJ kids/NNS)
(NP these/DT adults/NNS)
(NP a/DT k

(NP Star/NNP)
(NP the/DT kid/NN)
(NP screens/NNS)
(NP front/NN)
(NP ...../NNP)
(NP something/NN)
(NP i/NN)
(NP a/DT mortgage/NN)
(NP every/DT month/NN)
(NP year/NN)
(NP type/NN)
(NP newsdeck/NN)
(NP beck/NN)
(NP some/DT movie/NN)
(NP rich/JJ kids/NNS)
(NP some/DT bad/JJ terrible/JJ stuff/NN)
(NP A/NNP GIRL/NNP)
(NP JUST/NNP A/NNP GIRL/NNP)
(NP YOU/NNP)
(NP HECK/NNP)
(NP THEN/NNP THE/NNP)
(NP IS/NNP THE/NNP)
(NP DRIVER/NNP)
(NP LOL/NNP)
(NP RIM/NNP)
(NP SUMMER/NNP CAMP/NNP)
(NP Okay/NNP)
(NP Annabeth/NNP)
(NP X-Files/NNS)
(NP Oh/NNP)
(NP yeah/NN)
(NP a/DT small/JJ part/NN)
(NP tho/NN)
(NP the/DT rich/JJ brats/NNS)
(NP cool/NN)
(NP snap/NN)
(NP any/DT cool/JJ summer/NN)
(NP camps/NNS)
(NP i/NN)
(NP a/DT kid/NN)
(NP anyway/NN)
(NP The/DT alien/NN)
(NP critters/NNS)
(NP own/JJ dog/NN)
(NP critter/NN)
(NP backside/NN)
(NP a/DT kitchen/JJ scene/NN)
(NP Okay/NNP)
(NP purpose/NN)
(NP right/NN)
(NP cranial/JJ genitalia/NN)
(NP Star.Bahaha/NNP)
(NP THATS/NNP HAWT/NNP)
(NP year/NN)
(NP old/JJ gir

(NP films/NNS)
(NP first/JJ thing/NN)
(NP something/NN)
(NP the/DT other/JJ films/NNS)
(NP something/NN)
(NP hope/NN)
(NP The/DT film/NN)
(NP the/DT plot/NN)
(NP no/DT twists/NNS)
(NP the/DT plot/NN)
(NP The/DT special/JJ effects/NNS)
(NP this/DT film/NN)
(NP everything/NN)
(NP the/DT second/JJ film/NN)
(NP speaking/NN)
(NP accent/NN)
(NP the/DT words/NNS)
(NP the/DT freaking/JJ translation/NN)
(NP no/DT accent/NN)
(NP conclusion/NN)
(NP the/DT others/NNS)
(NP the/DT second/JJ film/NN)
(NP the/DT correct/JJ thing/NN)
(NP every/DT second/NN)
(NP Well/NNP)
(NP this/DT time/NN)

-----------------------********************************------------------
		 Rim of the World (2019)
-----------------------********************************------------------
(NP The/DT film/NN)
(NP the/DT fact/NN)
(NP Spielbergs/NNP)
(NP style/NN)
(NP the/DT polished/JJ finish/NN)
(NP Stangerthings/NNS)
(NP Attack/NNP)
(NP the/DT block/NN)
(NP The/DT fact/NN)
(NP adult/NN)
(NP jokes/NNS)
(NP the/DT movie/NN)
(NP 

(NP cartoon/NN)
(NP the/DT live/JJ action/NN)
(NP treatment/NN)
(NP the/DT backdrops/NNS)
(NP the/DT monkey/NN)
(NP good-looking/JJ lead/NN)
(NP way/NN)
(NP a/DT hackneyed/JJ plot/NN)
(NP roles/NNS)
(NP the/DT romantic/JJ interest/NN)
(NP the/DT evil/JJ black-clad/JJ villain/NN)
(NP no/DT depth/NN)
(NP intrigue/NN)
(NP sight/NN)
(NP The/DT action/NN)
(NP no/DT realism/NN)
(NP attention/NN)
(NP the/DT basics/NNS)
(NP physics/NNS)
(NP place/NN)
(NP interesting/JJ thing/NN)
(NP a/DT handful/NN)
(NP songs/NNS)
(NP another/DT wasted/JJ opportunity/NN)

-----------------------********************************------------------
		 Aladdin (2019)
-----------------------********************************------------------
(NP the/DT palace/NN)
(NP people/NNS)
(NP bread/NN)
(NP some/DT young/JJ urchins/NNS)
(NP no/DT money/NN)
(NP Street-wise/JJ thief/NN)
(NP the/DT rescue/NN)
(NP a/DT prince/NN)
(NP scheming/NN)
(NP advisor/NN)
(NP a/DT lamp/NN)
(NP nothing/NN)
(NP a/DT magic/JJ carpet/NN)
(NP the

(NP Aladdin/NNP)
(NP entertaining/JJ version/NN)
(NP Nights´/NNP)
(NP tale/NN)
(NP First/NNP)
(NP the/DT film/NN)
(NP the/DT role/NN)
(NP scenes/NNS)
(NP the/DT best/JJS moments/NNS)
(NP the/DT movie/NN)
(NP The/DT beauty/NN)
(NP shines/NNS)
(NP the/DT great/JJ villain/NN)
(NP good/JJ performance/NN)
(NP Aladdin/NNP)
(NP the/DT monkey/NN)
(NP the/DT evil/JJ macaw/NN)
(NP the/DT brilliant/JJ cast/NN)
(NP this/DT good/JJ version/NN)
(NP a/DT wonderful/JJ family/NN)
(NP entertainment/NN)
(NP vote/NN)
(NP Aladdin/NNP)

-----------------------********************************------------------
		 Aladdin (2019)
-----------------------********************************------------------
(NP Remember/NNP)
(NP reality/NN)
(NP TV/NN)
(NP shows/NNS)
(NP look/NN)
(NP an/DT idiot/NN)
(NP that/DT way/NN)
(NP this/DT film/NN)
(NP sang/NN)
(NP Hollywood/NNP)
(NP this/DT guy/NN)
(NP a/DT song/NN)
(NP high/JJ notes/NNS)
(NP way/NN)
(NP league/NN)
(NP the/DT story/NN)
(NP the/DT fact/NN)
(NP the/DT animati

...

(NP the/DT same/JJ creators/NNS)
(NP Despicable/JJ Me/NN)
(NP this/DT film/NN)
(NP the/DT first/JJ sequel/NN)
(NP that/DT same/JJ franchise/NN)
(NP fact/NN)
(NP Pets/NNS)
(NP a/DT notable/JJ success.The/NN)
(NP film/NN)
(NP the/DT characters/NNS)
(NP this/DT time/NN)
(NP owner/NN)
(NP little/JJ son/NN)
(NP a/DT long/JJ time/NN)
(NP fond/NN)
(NP the/DT child/NN)
(NP the/DT family/NN)
(NP a/DT season/NN)
(NP the/DT countryside/NN)
(NP the/DT opportunity/NN)
(NP fears/NNS)
(NP life/NN)
(NP a/DT pet/NN)
(NP the/DT white/JJ rabbit/NN)
(NP superhero/JJ illusions/NNS thanks/NNS)
(NP the/DT plays/NNS)
(NP child/NN)
(NP owner/NN)
(NP The/DT illusion/NN)
(NP the/DT chance/NN)
(NP a/DT young/JJ white/JJ tiger/NN)
(NP abuse/NN)
(NP the/DT hands/NNS)
(NP a/DT circus/NN)
(NP the/DT film/NN)
(NP several/JJ plots/NNS)
(NP parallel/NN)
(NP a/DT predominance/NN)
(NP the/DT plot/NN)
(NP the/DT tiger/NN)
(NP problem/NN)
(NP the/DT film/NN)
(NP central/JJ characters/NNS)
(NP the/DT previous/JJ film/NN)
(NP

(NP a/DT great/JJ opportunity/NN)
(NP D'Souza/NNP)
(NP fun/NN)
(NP superhero/NN)
(NP film/NN)
(NP a/DT message/NN)
(NP pollution/NN)
(NP brutal/JJ after-effects/NNS)
(NP the/DT film/NN)
(NP Synopsis/NN)
(NP A/DT timid/JJ young/JJ man/NN)
(NP gains/NNS superpowers/NNS)
(NP a/DT superhero/NN)
(NP a/DT mighty/NN)
(NP villain/NN)
(NP dirt/NN)
(NP pollution/NN)
(NP way/NN)
(NP a/DT few/JJ moments/NNS)
(NP the/DT protagonist/NN)
(NP banter/NN)
(NP boisterous/JJ mom/NN)
(NP bumbling/NN)
(NP brother/NN)
(NP The/DT superhero/NN)
(NP lack/NN)
(NP power/NN)
(NP an/DT unnecessary/JJ romantic/JJ angle/NN)
(NP nothing/NN)
(NP the/DT goings-on/NN)
(NP the/DT film/NN)
(NP a/DT spoilsport/NN)
(NP lacks/NNS)
(NP crispness/NN)
(NP Sachin-Jigar/NN)
(NP the/DT part/NN)
(NP the/DT rawness/NN)
(NP harder/NN)
(NP acting/NN)
(NP mom/NN)
(NP brother/NN)
(NP a/DT fine/JJ actor/NN)
(NP the/DT whole/NN)
(NP A/NNP Flying/NNP Jatt/NNP)
(NP stands/NNS)
(NP a/DT faulty/NN)
(NP script/NN)

-----------------------******

(NP Bollywood/NNP)
(NP choreographer/NN)
(NP films/NNS)
(NP a/DT superhero/NN)
(NP flick/NN)
(NP .Aman/NN)
(NP a/DT timid/JJ martial/JJ art/NN)
(NP teacher/NN)
(NP father/NN)
(NP afraid/NN)
(NP everything/NN)
(NP father/NN)
(NP a/DT numero-uno/JJ martial/JJ art/NN)
(NP Flying/NNP Jatt/NNP)
(NP the/DT righteousness/NN)
(NP community/NN)
(NP a/DT colony/NN)
(NP city/NN)
(NP pollution/NN)
(NP mother/NN)
(NP father/NN)
(NP the/DT super-power/NN)
(NP the/DT debris/NN)
(NP chemical/NN)
(NP factories/NNS)
(NP the/DT land/NN)
(NP community/NN)
(NP lives/NNS)
(NP people/NNS)
(NP powers/NNS)
(NP a/DT villain/NN)
(NP powerful.I/JJ wonder/NN)
(NP someone/NN)
(NP a/DT huge/JJ money/NN)
(NP tacky/JJ script/NN)
(NP a/DT wafer-thin/JJ plot/NN)
(NP the/DT execution/NN)
(NP many/JJ superhero/NN)
(NP movies/NNS)
(NP an/DT exact/JJ slo-mo/JJ scene/NN)
(NP X-Men/NN)
(NP Days/NNS)
(NP The/DT special/JJ effects/NNS)
(NP an/DT honest/JJ hack/NN)
(NP people/NNS)
(NP a/DT superhero/JJ movie/NN)
(NP superhero/NN

(NP Well/NNP)
(NP the/DT sense/NN)
(NP a/DT superhero/NN)
(NP costume/NN)
(NP some/DT punches/NNS)
(NP fights/NNS)
(NP the/DT top/JJ superhero/NN)
(NP flick/NN)
(NP different/JJ genres/NNS)
(NP these/DT films/NNS)
(NP clueless/NN)
(NP film/NN)
(NP makers/NNS)
(NP a/DT waste/NN)
(NP time/NN)
(NP deals/NNS)
(NP Religious/JJ superhero/NN)
(NP powers/NNS)
(NP villains/NNS)
(NP superhero/NN)
(NP the/DT villain/NN)
(NP the/DT story/NN)
(NP that.We/NN)
(NP Superheroes/NNP)
(NP a/DT passing/NN)
(NP mention/NN)
(NP Ra-One/NNP)
(NP Ra-one/NNP)
(NP Terminator/NNP)
(NP superheroes/NNS)
(NP a/DT tight/JJ script/NN)
(NP awesome/JJ action/NN)
(NP sequences/NNS)
(NP heart/NN)
(NP the/DT right/JJ place/NN)
(NP this/DT movie/NN)
(NP start/NN)
(NP the/DT stupidity/NN)
(NP super/JJ power/NN)
(NP a/DT person/NN)
(NP person/NN)
(NP abilities/NNS)
(NP HOW/NNP)
(NP powers/NNS)
(NP nature/NN)
(NP crime/NN)
(NP stupidity/NN)
(NP dust/NN)
(NP criminals/NNS)
(NP somehow/NN)
(NP a/DT rape/NN)
(NP victim/NN)
(NP th

(NP Views/NNS)
(NP An/DT average/JJ movie/NN)
(NP the/DT name/NN)
(NP a/DT hi-tech/JJ movie.Nice/NN)
(NP message/NN)
(NP the/DT issues/NNS)
(NP public/JJ concern/NN)
(NP real/JJ world/NN)
(NP a/DT scenario/NN)
(NP current/JJ material/NN)
(NP world/NN)
(NP decisions/NNS)
(NP children/NNS)
(NP This/DT kind/NN)
(NP situation/NN)
(NP many/JJ problems/NNS)
(NP the/DT children.learning/NN)
(NP youth/NN)
(NP any/DT phobia/NN)
(NP mind/NN)
(NP super/JJ skills/NNS)
(NP height/NN)
(NP doubt/NN)
(NP a/DT family/NN)
(NP movie/NN)
(NP a/DT value/NN)
(NP money/NN)
(NP movie/NN)
(NP shot/NN)
(NP video/NN)
(NP a/DT days/NNS)


The case study used movies from Comedy genre of IMDB.com. Reviews were used that were recorded on the rating scale of 1-10, where 1 indicates waste of time and 10 indicates Must watch. The 6 movies were selected manually and for each movie reviews was recorded. The total number of reviews found for these 6 movies count up for 150 reviews. 


